In [233]:
''' Manipulate Multi-Sample VCF files in Pandas (Python3). 
'''
from collections import OrderedDict
import pandas as pd
import re
import sys
import os
import collections

# path to this %%file
if sys.platform == "win32":
    file_path = os.path.dirname(os.path.abspath("__file__"))+"\\"
else:
    file_path = os.path.dirname(os.path.abspath("__file__"))+"/"

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)


def vcf2dataframe(filename, genotype_level=True, info_level=True, UID=False):
    '''Open a VCF file and returns a MultiIndex pandas.DataFrame.
    Args:
        genotype_level: place the genotype information into a second level column index
        info_level: place the info IDs into a second level column index
        UID: rename index to a unique variant identifier
    Notes:
        having any of these variables set to True will result
        in the DataFrame being generated very slowly. This is
        especially true for the UID variable.
    '''
    #if filename.endswith(".gz"):
    #    raise IOError("pdVCF does not support compressed VCF files.")

    # get INFO fields and Headers as lists
    VCF_HEADER = get_vcf_header(filename)
    INFO_FIELDS = get_info_fields(filename)

    # Count how many comment lines should be skipped.
    comments = count_comments(filename)

    # Return a simple dataframe representative of the VCF data.
    if filename.endswith(".gz"):
        df = pd.read_table(filename, skiprows=comments, compression='gzip', 
                           sep='\x01', names=VCF_HEADER, usecols=range(len(VCF_HEADER))) 
    else:
        df = pd.read_table(filename, skiprows=comments,
                           names=VCF_HEADER, usecols=range(len(VCF_HEADER)))

    if genotype_level:
        df = get_genotype_data(df)

    if info_level:
        df = get_info_data(df, INFO_FIELDS)

    if UID:
        df = index2UID(df)
    #else:
    #    df = df.set_index(['CHROM', 'POS', 'REF', 'ALT'])

    return df.drop('UID', axis=1)


def get_vcf_header(filename):
    ''' Get all header names from a given VCF file and return as a list.
    '''
    with open(filename) as input_file:
        row = [x for x in input_file if not x.startswith('##')] # skip unwanted headers
        head = next(iter(row))    # generator to deal with the header line only.
        split_head = [re.sub(r'#|\n', '', x) for x in head.split("\t")]
        return split_head


def get_info_fields(filename):
    ''' Get all ID names in the given VCFs INFO field and return as a list.
    '''
    with open(filename) as input_file:
        row = [x for x in input_file if x.startswith('##INFO')]
        info_fields = [x[11:].split(',')[0] for x in row]
        return info_fields



def count_comments(filename):
    ''' Count all lines in a given VCF file starting with #.
    '''
    comments = 0
    with open(filename) as f:
        for line in f:
            if line.startswith('#'):
                comments += 1

            else:
                break

    return comments



def replace_series_strings(df, col, dic, substring):
    ''' Replace the the keys with the items of the given
        dictionary for all strings or substrings in a
        given column
    Args:
        col: column name to replace strings
        dic: dictionary where the key is the string to replace with the item
        substrings: search and replace for either substrings (True) or exact strings (False)
    Returns:
        dataframe with the given column having all the
        entries identified as the key in the given dict
        replaced with the item in said dict
    '''
    if not isinstance(substring, bool):
        raise TypeError("substring argument must equal True or False")

    for string, correction in dic.items():
        if substring is True:
            df[col] = df[col].str.replace(string, correction)
        elif substring is False:
            df[col] = df[col].replace(string, correction, regex=True)

    return df



def get_genotype_data(df):
    ''' Give each sample column a second level column for every field
        detailed in the FORMAT column and return as a MultIndex dataframe.
    Args:
        df: DataFrame deriving from a VCF via vcf2dataframe()
    '''
    # contain the variant columns and the sample names in seperate lists
    normal = list(df.iloc[:, :9].columns)
    samples = list(df.iloc[:, 9:].columns)
    form = df['FORMAT'].str.split(":")[0]

    # These columns remain the same
    remain = pd.DataFrame(data=df[normal].values,
                          columns=pd.MultiIndex.from_tuples(
                            [(x, '') for x in normal] ))

    # list of dataframes where every sample has sub columns for each genotype info
    sams = [pd.DataFrame(data=list(df[col].str.split(':').dropna()),
                         columns=pd.MultiIndex.from_product([ [col], form ]))
            for col in samples]
    
    # add allele balance to sample genotype information
    sams = [calc_AB(sam) for sam in sams]
    
    # concat all dfs in the list
    df2 = pd.concat([remain] + sams, axis=1)

    return df2



def calc_AB(vcf):
    ''' Calculate allele balance for all samples in a given 
        pdVCF. Also converts DP & GQ to numeric type.
    
    Args:
        vcf: pdVCF with genotype information extracted
        
    Notes:
        ONLY WORKS FOR BIALLELIC ARIANTS
    '''
    sam = vcf.columns.levels[0][0]
    vcf[sam,'DP'] = pd.to_numeric(vcf[sam,'DP'])
    vcf[sam,'GQ'] = pd.to_numeric(vcf[sam,'GQ'])
    AD = vcf.xs('AD', level=1, axis=1).unstack().str.split(",", n=2)
    DP = vcf.xs('DP', level=1, axis=1).unstack()
    AB = round(pd.to_numeric(AD.str[1]) / pd.to_numeric(DP), 2)
    vcf[sam, 'AB'] = AB.tolist()
        
    return vcf



def get_info_data(df, info_fields):
    ''' Transform the INFO IDs into second level column indexes and return
        the df as a MultiIndex dataframe.
    Args:
        df: DataFrame deriving from a VCF via vcf2dataframe()
        info_fields: a list of all the INFO IDs in the given df
    '''
    # Alter Info field for some variables that don't work well
    df['INFO'] = df['INFO'].str.replace(";DB",";DB=1")
    df['INFO'] = df['INFO'].str.replace(";STR",";STR=1")

    # identify Info fields not present in each row and fill them with a 0
    for name in info_fields:
        if name == info_fields[0]:
            name = "{}=".format(name)
        else:
            name = ";{}=".format(name)

        not_present = df['INFO'][~df.INFO.str.contains(name)].add("{}0".format(name))
        present = df['INFO'][df.INFO.str.contains(name)]
        df['INFO'] = not_present.append(present).sort_index()

    # reorder INFO fields so they are are all in the same order
    df['INFO'] = df['INFO'].apply(lambda x: ';'.join(elem for elem in sorted(x.split(";"))))

    # remove all info_field names from the info values, starting with the info field with the longest name first
    unwanted = info_fields + ['=']
    unwanted.sort(key=len, reverse=True)
    remove = collections.OrderedDict([(x, '') for x in unwanted])
    df = replace_series_strings(df, col='INFO', dic=remove, substring=True)

    # create a new multi-index df containing only the info fields with the IDs as the second level
    info = pd.DataFrame(data=list(df['INFO'].str.split(';')),
                        columns=pd.MultiIndex.from_product([ ['INFO'], info_fields]))

    if not isinstance(df.columns, pd.MultiIndex):
        # create another multi-index df without the info fields where the second level is nothing
        df = pd.DataFrame(data=df.drop('INFO', axis=1).values,
                              columns=pd.MultiIndex.from_tuples(
                                [(x, '') for x in list(df.drop('INFO', axis=1).columns)] ))

    else:
        df = df.drop('INFO', axis=1)

    variant = df.iloc[:, :8]
    samples = df.iloc[:, 8:]

    # replace the info fields in the original df with the multi-index df created above
    final_df = pd.concat([variant] + [info] + [samples], axis=1)

    # MQ and MQ0 are in the wrong order so name swapping is required
    if 'MQ0' in info_fields and 'MQ' in info_fields:
        final_df = final_df.rename(columns={'MQ0': 'TEMP', 'MQ': 'MQ0'})
        final_df = final_df.rename(columns={'TEMP': 'MQ'})

    return final_df



def index2UID(df):
    ''' Replace the index with a unique variant identifier.
    '''
    if isinstance(df.columns, pd.MultiIndex):
        UID = df.apply(lambda x: "{}:{}-{}/{}".format(x['CHROM'][0], x['POS'][0],
                                                      x['REF'][0], x['ALT'][0]), axis=1)
    else:
         UID = df.apply(lambda x: "{}:{}-{}/{}".format(x['CHROM'], x['POS'],
                                                  x['REF'], x['ALT']), axis=1)

    df['UID'] = UID

    if df['UID'].value_counts()[0] > 1:
        raise ValueError("The UID is not unique.")

    return df.rename(UID)


In [2]:
j = "var.both.taadUkJan2017.filters.vcf"
o = "var.both.taadUkOctoberRepeatLibrariesDec2016.filters.vcf"
t = "26PL1207.oct.noInDels.recode.vcf"

In [255]:
import time
start = time.time()

vcf2dataframe(j, genotype_level=False, 
             info_level=False, UID=False)

done = time.time() - start
print(done)

1.1160786151885986


In [287]:
vcf = vcf2dataframe(o, genotype_level=True, 
             info_level=True, UID=True)

vcf[(vcf['21AI1224']['AB'] > 0.3) & (vcf['21AI1224']['DP'] > 49) & (vcf['21AI1224']['GQ'] > 29)]['21AI1224'].head()
vcf.head()


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:220: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:60: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.


CHROM      POS ID REF ALT     QUAL       FILTER          FORMAT  \
                                                                                
1:2160444-A/G  1     2160444  .  A   G   391.52   PASS         GT:AD:DP:GQ:PL   
1:2160448-G/A  1     2160448  .  G   A   162.5    QDfilterSNV  GT:AD:DP:GQ:PL   
1:2160449-C/T  1     2160449  .  C   T   5728.42  PASS         GT:AD:DP:GQ:PL   
1:2160451-G/A  1     2160451  .  G   A   969.15   PASS         GT:AD:DP:GQ:PL   
1:2160455-C/T  1     2160455  .  C   T   335.42   QDfilterSNV  GT:AD:DP:GQ:PL   

              INFO                                                             \
                AC     AF  AN BaseQRankSum DB    DP DS  Dels ExcessHet     FS   
1:2160444-A/G  15   0.341  44  -2.391       0  53    0  0.00  0.4046    0.000   
1:2160448-G/A  3    0.032  94  8.098        0  8099  0  0.00  3.1527    0.000   
1:2160449-C/T  1    0.010  96  22.044       0  8099  0  0.00  3.0103    0.000   
1:2160451-G/A  2    0.021  96  11.102       0  8099  0  0.00  3.0563    0.000   
1:2160455-C/T  1    0.010  96  7.644        0  8099  0  0.00  3.0103    0.000   

                                                                      \
              HaplotypeScore InbreedingCoeff MLEAC  MLEAF MQ0     MQ   
1:2160444-A/G  1.2037         -0.1143         12    0.273  0   59.44   
1:2160448-G/A  6.0582         -0.0364         2     0.021  0   60.00   
1:2160449-C/T  6.5337         -0.0148         1     0.010  0   60.00   
1:2160451-G/A  7.1673         -0.0294         1     0.010  0   60.00   
1:2160455-C/T  7.7508         -0.0148         1     0.010  0   60.00   

                                                                 \
              MQRankSum     QD RPA RU ReadPosRankSum    SOR STR   
1:2160444-A/G  0.000     27.96  0   0  -3.524         0.237  0    
1:2160448-G/A  0.001     0.40   0   0  0.073          0.000  0    
1:2160449-C/T  0.002     23.01  0   0  0.237          0.003  0    
1:2160451-G/A  0.001     2.09   0   0  0.108          0.000  0    
1:2160455-C/T  0.001     1.34   0   0  0.363          0.000  0    

                             21AI1224                                       \
                         set       GT     AD     DP    GQ          PL   AB   
1:2160444-A/G  variant        0/0      1,0    1.0    3.0   0,3,35      0.0   
1:2160448-G/A  FilteredInAll  0/0      249,0  250.0  99.0  0,155,1219  0.0   
1:2160449-C/T  variant        0/0      250,0  250.0  99.0  0,740,7485  0.0   
1:2160451-G/A  variant        0/0      250,0  250.0  99.0  0,140,1069  0.0   
1:2160455-C/T  FilteredInAll  0/0      250,0  250.0  99.0  0,722,7304  0.0   

              21IB1281                                       21IL1279         \
                    GT     AD     DP    GQ          PL    AB       GT     AD   
1:2160444-A/G  0/1      1,2    3.0    25.0  25,0,28     0.67  0/1      1,1     
1:2160448-G/A  0/0      135,0  135.0  75.0  0,75,578    0.00  0/0      249,1   
1:2160449-C/T  0/0      135,0  135.0  99.0  0,403,3973  0.00  0/0      250,0   
1:2160451-G/A  0/0      134,1  135.0  25.0  0,25,403    0.01  0/0      250,0   
1:2160455-C/T  0/0      135,0  135.0  99.0  0,400,3941  0.00  0/0      247,2   

                                             21JH1219                    \
                  DP    GQ          PL    AB       GT    AD    DP    GQ   
1:2160444-A/G  2.0    24.0  25,0,24     0.50  ./.      None NaN   NaN     
1:2160448-G/A  250.0  58.0  0,58,698    0.00  0/0      32,0  32.0  18.0   
1:2160449-C/T  250.0  99.0  0,743,7502  0.00  0/0      32,0  32.0  90.0   
1:2160451-G/A  250.0  69.0  0,69,528    0.00  0/0      32,0  32.0  89.0   
1:2160455-C/T  250.0  99.0  0,657,7161  0.01  0/0      32,0  32.0  90.0   

                             21JL1278                                        \
                     PL   AB       GT     AD     DP    GQ          PL    AB   
1:2160444-A/G  None     NaN   0/0      3,1    5.0    9.0   0,9,98      0.20   
1:2160448-G/A  0,18,142  0.0  0/

In [290]:

    

vcf = vcf2dataframe(t, genotype_level=True, 
              info_level=True, UID=True)
vcf

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:220: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:60: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.


CHROM      POS ID REF  ALT     QUAL       FILTER  \
                                                                   
1:1000-A/G       1     1000     .  A   G    24.89    LowQual       
1:2234385-C/T    1     2234385  .  C   T    9953.67  PASS          
1:2235243-C/T    1     2235243  .  C   T    82.06    PASS          
1:2235792-A/G    1     2235792  .  A   G    436.66   QDfilterSNV   
1:2235901-A/G    1     2235901  .  A   G    24.89    LowQual       
1:2239999-A/G,T  1     2239999  .  A   G,T  24.89    LowQual       

                         FORMAT INFO      26PL1207                   \
                                  AC LOLZ       GT      AD   DP  GQ   
1:1000-A/G       GT:AD:DP:GQ:PL  2    9    0/1      10,1    11   3    
1:2234385-C/T    GT:AD:DP:GQ:PL  1    90   0/1      5,11    16   99   
1:2235243-C/T    GT:AD:DP:GQ:PL  .0   0    0/1      4,1     5    16   
1:2235792-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      93,17   110  99   
1:2235901-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      10,1    11   3    
1:2239999-A/G,T  GT:AD:DP:GQ:PL  .0   0    0/1      10,1,4  15   3    

                                  22MI1099                           \
                         PL    AB       GT          AD     DP    GQ   
1:1000-A/G       3,0,197     0.09  ./.                 NaN    NaN     
1:2234385-C/T    289,0,103   0.69  0/1      10,1        11.0   3.0    
1:2235243-C/T    16,0,102    0.20  0/1      40,100      140.0  50.0   
1:2235792-A/G    145,0,1894  0.15  0/0      50,50       100.0  89.0   
1:2235901-A/G    3,0,197     0.09  1/1      200,200     400.0  90.0   
1:2239999-A/G,T  3,0,197     0.07  0/1      100,20,100  220.0  90.0   

                                      
                            PL    AB  
1:1000-A/G                     NaN    
1:2234385-C/T    3,0,197        0.09  
1:2235243-C/T    150,0,1888     0.71  
1:2235792-A/G    100,0,100      0.50  
1:2235901-A/G    100,0,100      0.50  
1:2239999-A/G,T  1000,500,1000  0.09

In [505]:

class FilterVCF(object):
    ''' A VCF file which can be readily filtered
    
    Attributes:
        vcf: Pandas DataFrame VCF
    '''
    def __init__(self, vcf):
        self.vcf = vcf

    def subset(self, sams, exclude_ref=False, remove_uncalled=True):
        ''' Subset a multisample VCF by a given samples.
        Args:
            vcf: Pandas DataFrame VCF
            sams: list of samples to subset the vcf for
            exlude_ref: remove variant if all GT values for subset are 0/0
            remove_uncalled: remove variant if all GT values for subset are ./.

        Returns:
            subsetted Pandas DataFrame VCF
        '''
        # split variant and genotype information 
        genotype = self.vcf[sams]
        num_info = self.vcf['INFO'].columns.shape[0]
        variant = self.vcf.ix[:,:8+num_info]

        GT = genotype.xs('GT', level=1, axis=1)
        uncalled= []

        if remove_uncalled:
            uncalled = GT[GT[sams] == './.'].dropna().index.tolist() 

        if exclude_ref:
            uncalled += GT[GT[sams] == '0/0'].dropna().index.tolist() 

        sub = pd.concat([variant, genotype], axis=1)
        self.vcf = sub.drop(uncalled)
        return self.vcf

    
    def genotype(self, minDP=None, minGQ=None, minAB=None):
        ''' Filter out variants in which all the samples in the given vcf 
            don't meet the minimum genotype values.
        
        Args:
            minDP: minimum variant depth
            minGQ: minimum genotype quality
            minAB: minimum allele balance
        '''
        # split variant and genotype information
        num_info = self.vcf['INFO'].columns.shape[0]
        variant = self.vcf.ix[:,:8+num_info]
        genotype = self.vcf.ix[:,9+num_info:]

        # store all variants that don't meet the minimum value given for the args here
        below_min = []
        
        if minDP:
            DP = genotype.xs('DP', level=1, axis=1).fillna(0)
            above_min = DP[DP > minDP] 
            below_min += DP[above_min.isnull().any(axis=1)].index.tolist()
            
        if minGQ:
            GQ = genotype.xs('GQ', level=1, axis=1).fillna(0)
            above_min = GQ[GQ > minGQ]
            below_min +=  GQ[above_min.isnull().any(axis=1)].index.tolist()
        
        if minAB:
            AB = genotype.xs('AB', level=1, axis=1).fillna(0)
            above_min = AB[AB > minAB]
            below_min +=  AB[above_min.isnull().any(axis=1)].index.tolist()
        
        # remove variants that don't meet the requirements from the vcf
        self.vcf = self.vcf.drop(below_min)
        return self.vcf
    


        
        

#subset(vcf, ['22MI1099', '26PL1207'], remove_uncalled=True, exclude_ref=True)

test_vcf = FilterVCF(vcf)
#test_vcf.vcf.ix[:,11:]
test_vcf.vcf

CHROM      POS ID REF  ALT     QUAL       FILTER  \
                                                                   
1:1000-A/G       1     1000     .  A   G    24.89    LowQual       
1:2234385-C/T    1     2234385  .  C   T    9953.67  PASS          
1:2235243-C/T    1     2235243  .  C   T    82.06    PASS          
1:2235792-A/G    1     2235792  .  A   G    436.66   QDfilterSNV   
1:2235901-A/G    1     2235901  .  A   G    24.89    LowQual       
1:2239999-A/G,T  1     2239999  .  A   G,T  24.89    LowQual       

                         FORMAT INFO      26PL1207                   \
                                  AC LOLZ       GT      AD   DP  GQ   
1:1000-A/G       GT:AD:DP:GQ:PL  2    9    0/1      10,1    11   3    
1:2234385-C/T    GT:AD:DP:GQ:PL  1    90   0/1      5,11    16   99   
1:2235243-C/T    GT:AD:DP:GQ:PL  .0   0    0/1      4,1     5    16   
1:2235792-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      93,17   110  99   
1:2235901-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      10,1    11   3    
1:2239999-A/G,T  GT:AD:DP:GQ:PL  .0   0    0/1      10,1,4  15   3    

                                  22MI1099                           \
                         PL    AB       GT          AD     DP    GQ   
1:1000-A/G       3,0,197     0.09  ./.                 NaN    NaN     
1:2234385-C/T    289,0,103   0.69  0/1      10,1        11.0   3.0    
1:2235243-C/T    16,0,102    0.20  0/1      40,100      140.0  50.0   
1:2235792-A/G    145,0,1894  0.15  0/0      50,50       100.0  89.0   
1:2235901-A/G    3,0,197     0.09  1/1      200,200     400.0  90.0   
1:2239999-A/G,T  3,0,197     0.07  0/1      100,20,100  220.0  90.0   

                                      
                            PL    AB  
1:1000-A/G                     NaN    
1:2234385-C/T    3,0,197        0.09  
1:2235243-C/T    150,0,1888     0.71  
1:2235792-A/G    100,0,100      0.50  
1:2235901-A/G    100,0,100      0.50  
1:2239999-A/G,T  1000,500,1000  0.09

In [432]:
test_vcf.subset(['22MI1099'], remove_uncalled=True, exclude_ref=True)
test_vcf.vcf


CHROM      POS ID REF  ALT     QUAL   FILTER          FORMAT  \
                                                                               
1:2234385-C/T    1     2234385  .  C   T    9953.67  PASS     GT:AD:DP:GQ:PL   
1:2235243-C/T    1     2235243  .  C   T    82.06    PASS     GT:AD:DP:GQ:PL   
1:2235901-A/G    1     2235901  .  A   G    24.89    LowQual  GT:AD:DP:GQ:PL   
1:2239999-A/G,T  1     2239999  .  A   G,T  24.89    LowQual  GT:AD:DP:GQ:PL   

                INFO      22MI1099                                          \
                  AC LOLZ       GT          AD     DP    GQ             PL   
1:2234385-C/T    1    90   0/1      10,1        11.0   3.0   3,0,197         
1:2235243-C/T    .0   0    0/1      40,100      140.0  50.0  150,0,1888      
1:2235901-A/G    .0   0    1/1      200,200     400.0  90.0  100,0,100       
1:2239999-A/G,T  .0   0    0/1      100,20,100  220.0  90.0  1000,500,1000   

                       
                   AB  
1:2234385-C/T    0.09  
1:2235243-C/T    0.71  
1:2235901-A/G    0.50  
1:2239999-A/G,T  0.09

In [504]:
test_vcf.genotype(minDP=10)#, minAB=0.1)
test_vcf.vcf#.loc[['1:1000-A/G']]


['1:1000-A/G', '1:2235243-C/T']


CHROM      POS ID REF  ALT     QUAL       FILTER  \
                                                                   
1:2234385-C/T    1     2234385  .  C   T    9953.67  PASS          
1:2235792-A/G    1     2235792  .  A   G    436.66   QDfilterSNV   
1:2235901-A/G    1     2235901  .  A   G    24.89    LowQual       
1:2239999-A/G,T  1     2239999  .  A   G,T  24.89    LowQual       

                         FORMAT INFO      26PL1207                   \
                                  AC LOLZ       GT      AD   DP  GQ   
1:2234385-C/T    GT:AD:DP:GQ:PL  1    90   0/1      5,11    16   99   
1:2235792-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      93,17   110  99   
1:2235901-A/G    GT:AD:DP:GQ:PL  .0   0    0/1      10,1    11   3    
1:2239999-A/G,T  GT:AD:DP:GQ:PL  .0   0    0/1      10,1,4  15   3    

                                  22MI1099                           \
                         PL    AB       GT          AD     DP    GQ   
1:2234385-C/T    289,0,103   0.69  0/1      10,1        11.0   3.0    
1:2235792-A/G    145,0,1894  0.15  0/0      50,50       100.0  89.0   
1:2235901-A/G    3,0,197     0.09  1/1      200,200     400.0  90.0   
1:2239999-A/G,T  3,0,197     0.07  0/1      100,20,100  220.0  90.0   

                                      
                            PL    AB  
1:2234385-C/T    3,0,197        0.09  
1:2235792-A/G    100,0,100      0.50  
1:2235901-A/G    100,0,100      0.50  
1:2239999-A/G,T  1000,500,1000  0.09